<a href="https://colab.research.google.com/github/kimhalyn/OPEN-API/blob/main/%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리
# colab에 xmltodict 라이브러리가 기본 설치되지 않으므로 별도 설치(외부 라이브러리)
!pip install xmltodict 
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from bs4 import BeautifulSoup
import pandas as pd
import requests, xmltodict, json

In [ ]:
# 판다스로 csv 파일 불러오기_iloc 사용
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 행,열만 추출
damcode = data.iloc[0:2,1]
#damcode = data.iloc[27,1]
for code in damcode:
  print(code)

2403201
2011602


[우량 관측소 조회]

In [ ]:
# 우량 관측소 코드 조회(1차)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/excll/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 댐코드 일부 추출
damcode = data.iloc[0:2,1]
df = pd.DataFrame()

for code in damcode:
  Params = '?' + urlencode({ quote_plus('ServiceKey') : key,
                                quote_plus('damcode') : code }) # 댐코드 입력
  # get 방식으로 api 호출
  res = requests.get(url + Params)

  # json으로 변환
  dict_res = xmltodict.parse(res.content)
  df = df.append(dict_res, ignore_index=True)
  #print(df)

  json_str = json.dumps(df['response']['body']['items'], ensure_ascii=False) # 값 없을때 에러
  json_obj = json.loads(json_str)

# csv파일로 저장
df = pd.DataFrame(json_obj['item'], index=[0])
df.to_csv('excllCode.csv' ,encoding='utf-8-sig')

KeyError: ignored

In [ ]:
# 우량 관측소 코드 조회(2차_에러처리)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/excll/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 행, 열만 추출(테스트)
damcode = data.iloc[0:2,1]

df = pd.DataFrame()

for code in damcode:
  Params = '?' + urlencode({ quote_plus('ServiceKey') : key,
                                quote_plus('damcode') : code }) # 댐코드 입력

  
  # get 방식으로 api 호출
  res = requests.get(url + Params).content
  #print(res)

  # json으로 변환
  dict_res = xmltodict.parse(res)
  print(dict_res)
  df = df.append(dict_res, ignore_index=True)

# 예외처리(실행할 코드, 예외발생 시, 예외발생 안 할때)
try:
  json_str = json.dumps(df['response']['body']['items'], ensure_ascii=False)
  json_obj = json.loads(json_str)
  
except Exception as ex:
  print('빈 결과', ex)
else:
   json_str = json.dumps(df['response']['body']['items'], ensure_ascii=False)
  json_obj = json.loads(json_str)


# csv파일로 저장
df = pd.DataFrame(json_obj['item'])
df.to_csv('001.csv' ,encoding='utf-8-sig')



In [ ]:
# 우량 관측소 코드 조회(3차_새 코드)
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from time import sleep

url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/excll/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 코드 조회
damcode = data.iloc[0:6,1]

dam = [] 
for code in damcode:

  Params = '?' + urlencode({quote_plus('ServiceKey') : key,
                                quote_plus('damcode') : code})
  
  #print(Params)
  res = requests.get(url + Params).text.encode('utf-8') # 요청에 대한 응답내용(.encode빼기도 함)
  sleep(0.5)
  #print(res) 
  xmlobj = bs4.BeautifulSoup(res, 'lxml') #lxml 파서(구문분석)  'lxml-xml'

  #item 태그 분리
  rows = xmlobj.findAll('item') # 매치되는 문자열을 리스트로 리턴
  #print(rows)

  # 반복문
  rowList = []
  nameList = []
  columnList = []

  rowsLen = len(rows)
  
  for i in range(0, rowsLen):

    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):      

      if i == 0:
        nameList.append(columns[j].name) # 칼럼명
      eachColumn = columns[j].text # 칼럼값
      columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []

  result = pd.DataFrame(rowList, columns=nameList)
  dam.append(result)




# csv파일로 저장
# df = pd.DataFrame(rowList, columns=nameList)
# df.to_csv("review.csv", encoding='utf-8-sig')

[]
0
[]
0
[]
0
[]
0
[<item><excllncobsrvtcode>9000040</excllncobsrvtcode><obsrvtnm>삼척시(광동댐)</obsrvtnm></item>, <item><excllncobsrvtcode>1001452</excllncobsrvtcode><obsrvtnm>태백시(상사미동)</obsrvtnm></item>]
2
[<excllncobsrvtcode>9000040</excllncobsrvtcode>, <obsrvtnm>삼척시(광동댐)</obsrvtnm>]
[<excllncobsrvtcode>1001452</excllncobsrvtcode>, <obsrvtnm>태백시(상사미동)</obsrvtnm>]
[]
0


In [ ]:
print(rows)


[]


[수위 관측소 조회]

In [ ]:
# 수위 관측소 코드 조회(원본)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/wal/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용
Params = '?' + urlencode({ quote_plus('ServiceKey') : key,
                                quote_plus('damcode') : '1012110' }) # 댐코드 입력

# get 방식으로 api 호출
res = requests.get(url + Params)

# json으로 변환
dict_res = xmltodict.parse(res.content)
json_str = json.dumps(dict_res['response']['body']['items'], ensure_ascii=False)
json_obj = json.loads(json_str)

# csv파일로 저장
df = pd.DataFrame(json_obj['item'])
df.to_csv('walCode.csv' ,encoding='utf-8-sig')

In [ ]:
# 수위 관측소 코드 조회(8.30_예외처리 전)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/wal/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 행, 열만 추출(테스트)
damcode = data.iloc[0:3,1]

df = pd.DataFrame()

for code in damcode:
  Params = '?' + urlencode({ quote_plus('ServiceKey') : key,
                                quote_plus('damcode') : code }) # 댐코드 입력
  print(code)
  print(Params)
  
  # get 방식으로 api 호출
  res = requests.get(url + Params)

  # json으로 변환
  dict_res = xmltodict.parse(res.content)
  print(dict_res)
  df = df.append(dict_res, ignore_index=True)

  print(df)
  json_str = json.dumps(dict_res['response']['body']['items'], ensure_ascii=False)
  json_obj = json.loads(json_str)

# csv파일로 저장
df = pd.DataFrame(json_obj['item'])
df.to_csv('예외처리전.csv' ,encoding='utf-8-sig')

2403201
?ServiceKey=jkd4PnVjssNP%2Bg2WeRzs8lgc4SOelLsncvuMr81%2F9R9nSYKkiQAcb2xt8GTYgTJSQGQ4%2BhtuRuyIXSwRXr210w%3D%3D&damcode=2403201
OrderedDict([('response', OrderedDict([('header', OrderedDict([('resultCode', '00'), ('resultMsg', 'NORMAL SERVICE.')])), ('body', OrderedDict([('items', None)]))]))])
                                            response
0  {'header': {'resultCode': '00', 'resultMsg': '...
2011602
?ServiceKey=jkd4PnVjssNP%2Bg2WeRzs8lgc4SOelLsncvuMr81%2F9R9nSYKkiQAcb2xt8GTYgTJSQGQ4%2BhtuRuyIXSwRXr210w%3D%3D&damcode=2011602
OrderedDict([('response', OrderedDict([('header', OrderedDict([('resultCode', '00'), ('resultMsg', 'NORMAL SERVICE.')])), ('body', OrderedDict([('items', OrderedDict([('item', [OrderedDict([('obsrvtNm', '강정고령보상류'), ('walobsrvtcode', '2011602')]), OrderedDict([('obsrvtNm', '강정고령보하류'), ('walobsrvtcode', '2011655')])])]))]))]))])
                                            response
0  {'header': {'resultCode': '00', 'resultMsg': '...
1  {'header': {'resul

KeyError: ignored

In [ ]:
# 수위 관측소 코드 조회(8.30_예외처리 전)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/wal/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 행, 열만 추출(테스트)
damcode = data.iloc[0:3,1]

df = pd.DataFrame()

for code in damcode:
  Params = '?' + urlencode({ quote_plus('ServiceKey') : key,
                                quote_plus('damcode') : code }) # 댐코드 입력
  print(code)
  print(Params)
  
  # get 방식으로 api 호출
  res = requests.get(url + Params)

  # json으로 변환
  dict_res = xmltodict.parse(res.content)
  print(dict_res)
  df = df.append(dict_res, ignore_index=True)

  print(df)
  json_str = json.dumps(dict_res['response']['body'], ensure_ascii=False)
  json_obj = json.loads(json_str)

  for item in json_obj['items']['item']:
    df = pd.DataFrame(json_obj['item'])
    df.to_csv('예외처리전.csv' ,encoding='utf-8-sig')


# csv파일로 저장


2403201
?ServiceKey=jkd4PnVjssNP%2Bg2WeRzs8lgc4SOelLsncvuMr81%2F9R9nSYKkiQAcb2xt8GTYgTJSQGQ4%2BhtuRuyIXSwRXr210w%3D%3D&damcode=2403201
OrderedDict([('response', OrderedDict([('header', OrderedDict([('resultCode', '00'), ('resultMsg', 'NORMAL SERVICE.')])), ('body', OrderedDict([('items', None)]))]))])
                                            response
0  {'header': {'resultCode': '00', 'resultMsg': '...


TypeError: ignored

예외처리
https://responding.tistory.com/47

In [ ]:
# 수위 관측소 코드 조회(예외처리1)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/wal/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 행, 열만 추출(테스트)
damcode = data.iloc[0:3,1]

df = pd.DataFrame()

for code in damcode:
  Params = '?' + urlencode({ quote_plus('ServiceKey') : key,
                                quote_plus('damcode') : code }) # 댐코드 입력

  
  # get 방식으로 api 호출
  res = requests.get(url + Params)
  print(res)

  # json으로 변환
  dict_res = xmltodict.parse(res.content)
  print(dict_res)
  df = df.append(dict_res, ignore_index=True)


# 예외처리(실행할 코드, 예외발생 시, 예외발생 안 할때)
try:
  json_str = json.dumps(df['response']['body']['items'], ensure_ascii=False)
  json_obj = json.loads(json_str)
  
except Exception as ex:
  print('빈 결과', ex)

else:
  json_str = json.dumps(df['response']['body']['items'], ensure_ascii=False)
  json_obj = json.loads(json_str)


# csv파일로 저장
df = pd.DataFrame(json_obj['item'])
df.to_csv('001.csv' ,encoding='utf-8-sig')

<Response [200]>
OrderedDict([('response', OrderedDict([('header', OrderedDict([('resultCode', '00'), ('resultMsg', 'NORMAL SERVICE.')])), ('body', OrderedDict([('items', None)]))]))])
<Response [200]>
OrderedDict([('response', OrderedDict([('header', OrderedDict([('resultCode', '99'), ('resultMsg', 'LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.')]))]))])
<Response [200]>
OrderedDict([('response', OrderedDict([('header', OrderedDict([('resultCode', '00'), ('resultMsg', 'NORMAL SERVICE.')])), ('body', OrderedDict([('items', OrderedDict([('item', [OrderedDict([('obsrvtNm', '강천보상류'), ('walobsrvtcode', '1007601')]), OrderedDict([('obsrvtNm', '강천보하류'), ('walobsrvtcode', '1005681')])])]))]))]))])
빈 결과 'body'


KeyError: ignored

Parser: parsing을 수행하는 프로그램
원시 프로그램을 읽어 들여 그 문장의 구조를 알아내는 **parsing(구문 분석)**을 행하는 프로그램



BeautifulSoup: HTML 문서를 탐색해서 원하는 부분만 쉽게 뽑아낼 수 있는 파이썬 라이브러리 

수정2 코드 전문

https://nicola-ml.tistory.com/97

https://greendreamtrre.tistory.com/268

In [ ]:
# 수위 관측소 코드 조회(전체 수정2)원본
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/wal/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 코드 조회
damcode = data.iloc[0:3,1]

for code in damcode:

  Params = '?' + urlencode({quote_plus('ServiceKey') : key,
                                quote_plus('damcode') : code})
  
  #print(Params)
  res = requests.get(url + Params).text.encode('utf-8') # 요청에 대한 응답내용(.encode빼기도 함)
  print(res) #문제
  xmlobj = bs4.BeautifulSoup(res, 'lxml') #lxml 파서(구문분석)  'lxml-xml'

  #item 태그 분리
  rows = xmlobj.findAll('item') # 매치되는 문자열을 리스트로 리턴
  #print(rows)

  # 반복문
  rowList = []
  nameList = []
  columnList = []

  rowsLen = len(rows) #len(): 원소 개수
  for i in range(0, rowsLen):
    columns = rows[i].find_all()
    columnsLen = len(columns)
    for j in range(0, columnsLen):
      

      # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
      if i == 0:
        nameList.append(columns[j].name)
      # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)


# csv파일로 저장
df = pd.DataFrame(rowList, columns=nameList)
df.to_csv('수위_방법2.csv' ,encoding='utf-8-sig')

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultMsg></header></response>'
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultMsg></header></response>'
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultMsg></header></response>'


[우량 분강우량]

In [ ]:
# (1)우량 분강우량 조회_1(10분 단위)_성공(원본)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/mntrf/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용
Params = '?' + urlencode({ quote_plus('ServiceKey') : key ,
                                quote_plus('sdate') : '2018-01-01',
                                quote_plus('stime') : '00',
                                quote_plus('edate') : '2018-12-31',
                                quote_plus('etime') : '24',
                                quote_plus('excll') : '1012432', # 우량관측소 코드
                                quote_plus('tms') : '10',
                                quote_plus('numOfRows') : '80000',
                                quote_plus('pageNo') : '1' })

# get 방식으로 api 호출
res = requests.get(url + Params).content

# json으로 변환
dict_res = xmltodict.parse(res)
json_str = json.dumps(dict_res['response']['body']['items'], ensure_ascii=False)
json_obj = json.loads(json_str)
#print(json_obj)

# csv파일로 저장
df = pd.DataFrame(json_obj['item'])
df.to_csv('sample_9.csv' ,encoding='utf-8-sig')

In [ ]:
# (1)우량 분강우량 조회_1(10분 단위)_성공(반복문 추가)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/mntrf/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용


for excll in excllcode:
  Params = '?' + urlencode({ quote_plus('ServiceKey') : key ,
                              quote_plus('sdate') : '2018-01-01',
                              quote_plus('stime') : '00',
                              quote_plus('edate') : '2018-12-31',
                              quote_plus('etime') : '24',
                              quote_plus('excll') : '1012432', # 우량관측소 코드
                              quote_plus('tms') : '10',
                              quote_plus('numOfRows') : '80000',
                              quote_plus('pageNo') : '1' })

  # get 방식으로 api 호출
  res = requests.get(url + Params).content

  # json으로 변환
  dict_res = xmltodict.parse(res)
  json_str = json.dumps(dict_res['response']['body']['items'], ensure_ascii=False)
  json_obj = json.loads(json_str)
  #print(json_obj)

# csv파일로 저장
df = pd.DataFrame(json_obj['item'])
df.to_csv('sample_9.csv' ,encoding='utf-8-sig')

[수위 분강우량]

In [ ]:
## (4)수위 분강우량 조회_1(10분 단위)(원본)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/mntwal/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용
Params = '?' + urlencode({ quote_plus('ServiceKey') : key ,
                                quote_plus('sdate') : '2014-12-01',
                                quote_plus('stime') : '08',
                                quote_plus('edate') : '2014-12-01',
                                quote_plus('etime') : '15',
                                quote_plus('damcode') : '15', # 댐코드
                                quote_plus('wal') : '1012432', # 수위 관측소 코드
                                quote_plus('tms') : '10',
                                quote_plus('numOfRows') : '10',
                                quote_plus('pageNo') : '1' })

# get 방식으로 api 호출
res = requests.get(url + Params)

# json으로 변환
dict_res = xmltodict.parse(res.content)
json_str = json.dumps(dict_res['response']['body']['items'], ensure_ascii=False)
json_obj = json.loads(json_str)

# csv파일로 저장
df = pd.DataFrame(json_obj['item'])
df.to_csv('mntwal_10.csv' ,encoding='utf-8-sig')

In [ ]:
## (4)수위 분강우량 조회_1(10분 단위)(반복문 추가)
url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/mntwal/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' #디코딩 사용

for code in damcode:
  for excll in excllcode:
    
    Params = '?' + urlencode({ quote_plus('ServiceKey') : key ,
                                    quote_plus('sdate') : '2014-12-01',
                                    quote_plus('stime') : '08',
                                    quote_plus('edate') : '2014-12-01',
                                    quote_plus('etime') : '15',
                                    quote_plus('damcode') : '15', # 댐코드
                                    quote_plus('wal') : '1012432', # 수위 관측소 코드
                                    quote_plus('tms') : '10',
                                    quote_plus('numOfRows') : '10',
                                    quote_plus('pageNo') : '1' })

    # get 방식으로 api 호출
    res = requests.get(url + Params)

    # json으로 변환
    dict_res = xmltodict.parse(res.content)
    json_str = json.dumps(dict_res['response']['body']['items'], ensure_ascii=False)
    json_obj = json.loads(json_str)

# csv파일로 저장
df = pd.DataFrame(json_obj['item'])
df.to_csv('mntwal_10.csv' ,encoding='utf-8-sig')
